In [1]:
import numpy as np
import pandas as pd
from scipy.stats import zscore

data = pd.read_parquet('/content/drive/MyDrive/data.parquet')
data = data.between_time('09:15', '15:30').copy()

missing_values = data.isnull().sum()


if missing_values.sum() > 0:
    data_filled = data.fillna(method='ffill')
else:
    data_filled = data

data_filled['spread'] = data_filled['banknifty'] - data_filled['nifty']


missing_values, data_filled.head()


data_filled['z_score'] = zscore(data_filled['spread'])

data_filled.head()


ENTRY_THRESHOLD = 2.0 # Enter trade if z-score is above +2 or below -2
EXIT_THRESHOLD = 1  # Exit trade if z-score comes within +1 or -1


data_filled['long_entry'] = data_filled['z_score'] < -ENTRY_THRESHOLD
data_filled['short_entry'] = data_filled['z_score'] > ENTRY_THRESHOLD
data_filled['long_exit'] = data_filled['z_score'] > -EXIT_THRESHOLD
data_filled['short_exit'] = data_filled['z_score'] < EXIT_THRESHOLD


long_positions = np.zeros(len(data_filled))
short_positions = np.zeros(len(data_filled))


for i in range(1, len(data_filled)):

    if data_filled['long_entry'][i] and not long_positions[i-1]:
        long_positions[i] = 1
    elif data_filled['long_exit'][i] and long_positions[i-1]:
        long_positions[i] = 0
    else:
        long_positions[i] = long_positions[i-1]


    if data_filled['short_entry'][i] and not short_positions[i-1]:
        short_positions[i] = -1
    elif data_filled['short_exit'][i] and short_positions[i-1]:
        short_positions[i] = 0
    else:
        short_positions[i] = short_positions[i-1]


data_filled['long_positions'] = long_positions
data_filled['short_positions'] = short_positions
data_filled['positions'] = data_filled['long_positions'] + data_filled['short_positions']


data_filled['pnl'] = data_filled['spread'].diff() * data_filled['positions'].shift(1)

data_filled['pnl'] *= (data_filled['tte'] ** 0.7)

data_filled.dropna(inplace=True)

data_filled['cum_pnl'] = data_filled['pnl'].cumsum()

sharpe_ratio = data_filled['pnl'].mean() / data_filled['pnl'].std() * np.sqrt(252 * 375) # 252 trading days, 375 minutes per day


roll_max = data_filled['cum_pnl'].cummax()
daily_drawdown = data_filled['cum_pnl'] - roll_max
max_drawdown = daily_drawdown.min()


In [2]:
data_filled.head()

,banknifty,nifty,tte,spread,z_score,long_entry,short_entry,long_exit,short_exit,long_positions,short_positions,positions,pnl,cum_pnl
time,,,,,,,,,,,,,,
2021-01-01 09:16:00,0.285381,0.200433,27,0.084948,0.491736,False,False,True,True,0.0,0.0,0.0,-0.0,-0.0
2021-01-01 09:17:00,0.284233,0.200004,27,0.084229,0.464628,False,False,True,True,0.0,0.0,0.0,-0.0,-0.0
2021-01-01 09:18:00,0.286104,0.199860,27,0.086244,0.540526,False,False,True,True,0.0,0.0,0.0,0.0,0.0
2021-01-01 09:19:00,0.285539,0.198951,27,0.086588,0.553505,False,False,True,True,0.0,0.0,0.0,0.0,0.0
2021-01-01 09:20:00,0.283431,0.198716,27,0.084715,0.482938,False,False,True,True,0.0,0.0,0.0,-0.0,0.0


In [3]:
sharpe_ratio

5.15706321765083

In [4]:
max_drawdown

-3.3477966500492595

In [10]:
import pandas as pd
import numpy as np


data = pd.read_parquet('/content/drive/MyDrive/data.parquet')
data = data.between_time('09:15', '15:30').copy()

data['banknifty'].fillna(method='ffill', inplace=True)
data['nifty'].fillna(method='ffill', inplace=True)

data['spread'] = data['banknifty'] - data['nifty']

data['P/L'] = data['spread'] * (data['tte'] ** 0.7)

# Define parameters for the moving average and the entry threshold
lookback_period = 60  # lookback period for moving average in minutes
entry_threshold = 2   # number of standard deviations for entry signal


data['moving_average'] = data['spread'].rolling(window=lookback_period, min_periods=1).mean()
data['moving_std_dev'] = data['spread'].rolling(window=lookback_period, min_periods=1).std()


data['long_entry'] = data['spread'] < (data['moving_average'] - entry_threshold * data['moving_std_dev'])
data['short_entry'] = data['spread'] > (data['moving_average'] + entry_threshold * data['moving_std_dev'])


exit_after_periods = 30  # exit after this many minutes
data['exit_long'] = data['long_entry'].shift(periods=exit_after_periods).fillna(False)
data['exit_short'] = data['short_entry'].shift(periods=exit_after_periods).fillna(False)


data['position'] = 0
data.loc[data['long_entry'], 'position'] = 1
data.loc[data['short_entry'], 'position'] = -1
data.loc[data['exit_long'], 'position'] = 0
data.loc[data['exit_short'], 'position'] = 0
data['position'] = data['position'].ffill()
data.loc[data['exit_long'] | data['exit_short'], 'position'] = 0


data['strategy_return'] = data['position'].shift(1) * data['spread'].diff()

data['cumulative_pl'] = data['strategy_return'].cumsum()


sharpe_ratio = data['strategy_return'].mean() / data['strategy_return'].std() * np.sqrt(252 * 375)
rolling_max = data['cumulative_pl'].cummax()
daily_drawdown = data['cumulative_pl'] - rolling_max
max_drawdown = daily_drawdown.min()


In [11]:
sharpe_ratio

16.604970495256023

In [12]:
max_drawdown

-0.04514550000000028

In [13]:
data

,banknifty,nifty,tte,spread,P/L,moving_average,moving_std_dev,long_entry,short_entry,exit_long,exit_short,position,strategy_return,cumulative_pl
time,,,,,,,,,,,,,,
2021-01-01 09:15:00,0.286058,0.199729,27,0.086329,0.867184,0.086329,NaN,False,False,False,False,0,NaN,NaN
2021-01-01 09:16:00,0.285381,0.200433,27,0.084948,0.853317,0.085639,0.000976,False,False,False,False,0,-0.0,0.000000
2021-01-01 09:17:00,0.284233,0.200004,27,0.084229,0.846089,0.085169,0.001067,False,False,False,False,0,-0.0,0.000000
2021-01-01 09:18:00,0.286104,0.199860,27,0.086244,0.866325,0.085437,0.001024,False,False,False,False,0,0.0,0.000000
2021-01-01 09:19:00,0.285539,0.198951,27,0.086588,0.869786,0.085668,0.001025,False,False,False,False,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 15:26:00,0.240701,0.214758,28,0.025943,0.267320,0.026049,0.001027,False,False,False,False,0,0.0,14.609908
2022-06-30 15:27:00,0.240875,0.216558,28,0.024317,0.250560,0.026037,0.001042,False,False,False,False,0,-0.0,14.609908
2022-06-30 15:28:00,0.242115,0.216794,28,0.025321,0.260910,0.026049,0.001029,False,False,False,False,0,0.0,14.609908


In [8]:
import numpy as np
import pandas as pd
from scipy.stats import norm


data = pd.read_parquet('/content/drive/MyDrive/data.parquet')
data = data.between_time('09:15', '15:30').copy()


data['banknifty'].fillna(method='ffill', inplace=True)
data['nifty'].fillna(method='ffill', inplace=True)


data['spread'] = data['banknifty'] - data['nifty']


data['P/L'] = data['spread'] * (data['tte'] ** 0.7)


lookback_period = 60  # lookback period for moving average in minutes


data['moving_average'] = data['spread'].rolling(window=lookback_period, min_periods=1).mean()
data['moving_std_dev'] = data['spread'].rolling(window=lookback_period, min_periods=1).std()


exit_after_periods = 30  # exit after this many minutes


results = pd.DataFrame(columns=['Std_Dev_Threshold', 'Sharpe_Ratio', 'Cumulative_PL', 'Max_Drawdown'])


for entry_threshold in np.linspace(1, 4, 15):


    data['long_entry'] = data['spread'] < (data['moving_average'] - entry_threshold * data['moving_std_dev'])
    data['short_entry'] = data['spread'] > (data['moving_average'] + entry_threshold * data['moving_std_dev'])


    data['position'] = np.where(data['long_entry'], 1, np.where(data['short_entry'], -1, 0))
    data['position'] = data['position'].replace(to_replace=0, method='ffill', limit=exit_after_periods)
    data.loc[data['position'].shift(1) != 0, 'position'] = 0


    data['strategy_return'] = data['position'].shift(1) * data['spread'].diff()

    data['cumulative_pl'] = data['strategy_return'].cumsum()

    sharpe_ratio = np.sqrt(252 * 375) * data['strategy_return'].mean() / data['strategy_return'].std()
    rolling_max = data['cumulative_pl'].cummax()
    daily_drawdown = data['cumulative_pl'] - rolling_max
    max_drawdown = daily_drawdown.min()

    new_row = pd.DataFrame({
        'Std_Dev_Threshold': entry_threshold,
        'Sharpe_Ratio': sharpe_ratio,
        'Cumulative_PL': data['cumulative_pl'].iloc[-1],
        'Max_Drawdown': max_drawdown }, index=[0])
    results = pd.concat([results, new_row], ignore_index=True)

optimal_threshold = results.loc[results['Sharpe_Ratio'].idxmax()]

optimal_threshold


Std_Dev_Threshold    2.285714
Sharpe_Ratio         6.708494
Cumulative_PL        2.414162
Max_Drawdown        -0.046615
Name: 6, dtype: float64

In [9]:
data

,banknifty,nifty,tte,spread,P/L,moving_average,moving_std_dev,long_entry,short_entry,position,strategy_return,cumulative_pl
time,,,,,,,,,,,,
2021-01-01 09:15:00,0.286058,0.199729,27,0.086329,0.867184,0.086329,NaN,False,False,0,NaN,NaN
2021-01-01 09:16:00,0.285381,0.200433,27,0.084948,0.853317,0.085639,0.000976,False,False,0,-0.0,0.000000
2021-01-01 09:17:00,0.284233,0.200004,27,0.084229,0.846089,0.085169,0.001067,False,False,0,-0.0,0.000000
2021-01-01 09:18:00,0.286104,0.199860,27,0.086244,0.866325,0.085437,0.001024,False,False,0,0.0,0.000000
2021-01-01 09:19:00,0.285539,0.198951,27,0.086588,0.869786,0.085668,0.001025,False,False,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 15:26:00,0.240701,0.214758,28,0.025943,0.267320,0.026049,0.001027,False,False,0,0.0,0.711813
2022-06-30 15:27:00,0.240875,0.216558,28,0.024317,0.250560,0.026037,0.001042,False,False,0,-0.0,0.711813
2022-06-30 15:28:00,0.242115,0.216794,28,0.025321,0.260910,0.026049,0.001029,False,False,0,0.0,0.711813
